In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pytickersymbols import PyTickerSymbols

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
#This Cell Block is all the functions for the strat. This is the only cell block that needs to be changed to backtest a different strategy.
def RSI(data, window=14):
    #daily_data = data.resample('D').last()
    
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

def strat(ticker, period):   
    # Download stock data from Yahoo Finance
    data = yf.download(ticker, period = period, interval = "1d")
    
    # Calculate RSI and add it to the DataFrame
    data['RSI'] = RSI(data)
    
    # Define the buy and sell conditions based on RSI
    data['Signal'] = 0
    data['Signal'][data['RSI'] > 53] = 1   # Buy signal
    data['Signal'][data['RSI'] > 63] = -1  # Sell signal
    
    # Shift signals so we enter positions on the next day
    data['Position'] = data['Signal'].shift(1)

    # Initialize a holding counter
    holding_counter = 0

    data['Hold'] = 0
    
    # Loop through the data to check conditions and manage holding duration
    for i in range(1, len(data)):
        if data['Position'].iloc[i-1] == 1:  # Buy condition
            holding_counter += 1
            data['Hold'].iloc[i] = 1  # Hold the stock
        elif data['Position'].iloc[i-1] == -1 or holding_counter >= 5:  # Sell condition or 5-day hold
            data['Hold'].iloc[i] = 0  # Sell the stock
            holding_counter = 0  # Reset counter after selling
        else:
            data['Hold'].iloc[i] = data['Hold'].iloc[i-1]  # Continue holding
            
    
    
    # Calculate daily returns
    data['Daily Return'] = data['Close'].pct_change()
    # Calculate strategy returns: Position * Daily Return
    data['Strategy Return'] = data['Position'] * data['Daily Return']
    
    # Calculate cumulative returns
    data['Cumulative Market Return'] = (1 + data['Daily Return']).cumprod() - 1
    data['Cumulative Strategy Return'] = (1 + data['Strategy Return']).cumprod() - 1

    total_strategy_return = data['Cumulative Strategy Return'].iloc[-1] * 100
    total_strategy_return = "{:.2f}%".format(total_strategy_return)

    print(data['Position'])
    return total_strategy_return

In [3]:
#The backtesting function to backtest the strat over different periods
def backtest_strat(ticker):
    backtest_data = {
        'Asset': [ticker],
        'YTD': [strat(ticker, "ytd")],
        'Whole Period': [strat(ticker, "max")],
        'Last 10Y': [strat(ticker, "10y")],
        'Last 5Y': [strat(ticker, "5y")],
        'Last 2Y': [strat(ticker, "2y")],
        'Last 1Y': [strat(ticker, "1y")],
        'Last 6M': [strat(ticker, "6mo")],
        'Last 3M': [strat(ticker, "3mo")],
        'Last 1M': [strat(ticker, "1mo")]    }
    
    # Create a pandas dataframe
    backtest_df = pd.DataFrame(backtest_data)
    
    # Set the first column as the index for clarity
    backtest_df.set_index('Asset', inplace=True)
    return backtest_df

In [4]:
#Creates an empty dataframe to store the backtest data
df =pd.DataFrame({
        'Asset': [],'YTD': [], 'Whole Period': [], 'Last 10Y': [], 'Last 5Y': [], 'Last 2Y': [], 'Last 1Y': [], 'Last 6M': [], 'Last 3M': [], 'Last 1M': []
    })

df.set_index('Asset', inplace=True)

In [5]:
#Fetching the tickers for the S&P 500
#This will take like 10mins to execute
stock_data = PyTickerSymbols()
sp500_symbols = stock_data.get_stocks_by_index('S&P 500')

#symbols = [stock['symbol'] for stock in sp500_symbols]
symbols = ["AAPL", "NVDA"] #Test data

#Running the stocks through the strat and backtesting functions
for symbol in symbols:
    try:
        df = pd.concat([df, backtest_strat(symbol)])
    except IndexError:
        pass

df

[*********************100%***********************]  1 of 1 completed


Date
2024-01-02    NaN
2024-01-03    0.0
2024-01-04    0.0
2024-01-05    0.0
2024-01-08    0.0
             ... 
2024-10-11    0.0
2024-10-14    0.0
2024-10-15    1.0
2024-10-16    1.0
2024-10-17    1.0
Name: Position, Length: 201, dtype: float64


[*********************100%***********************]  1 of 1 completed


Date
1980-12-12    NaN
1980-12-15    0.0
1980-12-16    0.0
1980-12-17    0.0
1980-12-18    0.0
             ... 
2024-10-11    0.0
2024-10-14    0.0
2024-10-15    1.0
2024-10-16    1.0
2024-10-17    1.0
Name: Position, Length: 11054, dtype: float64


[*********************100%***********************]  1 of 1 completed


...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Date
2023-10-17    NaN
2023-10-18    0.0
2023-10-19    0.0
2023-10-20    0.0
2023-10-23    0.0
             ... 
2024-10-11   -1.0
2024-10-14   -1.0
2024-10-15   -1.0
2024-10-16    1.0
2024-10-17   -1.0
Name: Position, Length: 253, dtype: float64
Date
2024-04-17    NaN
2024-04-18    0.0
2024-04-19   -1.0
2024-04-22    0.0
2024-04-23    0.0
             ... 
2024-10-11   -1.0
2024-10-14   -1.0
2024-10-15   -1.0
2024-10-16    1.0
2024-10-17   -1.0
Name: Position, Length: 128, dtype: float64


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Date
2024-07-17    NaN
2024-07-18    0.0
2024-07-19   -1.0
2024-07-22    0.0
2024-07-23   -1.0
             ... 
2024-10-11   -1.0
2024-10-14   -1.0
2024-10-15   -1.0
2024-10-16    1.0
2024-10-17   -1.0
Name: Position, Length: 66, dtype: float64
Date
2024-09-17    NaN
2024-09-18    0.0
2024-09-19    0.0
2024-09-20   -1.0
2024-09-23    0.0
2024-09-24    1.0
2024-09-25   -1.0
2024-09-26   -1.0
2024-09-27   -1.0
2024-09-30   -1.0
2024-10-01   -1.0
2024-10-02    0.0
2024-10-03    1.0
2024-10-04    1.0
2024-10-07   -1.0
2024-10-08   -1.0
2024-10-09   -1.0
2024-10-10   -1.0
2024-10-11   -1.0
2024-10-14   -1.0
2024-10-15   -1.0
2024-10-16    1.0
2024-10-17   -1.0
Name: Position, dtype: float64


,YTD,Whole Period,Last 10Y,Last 5Y,Last 2Y,Last 1Y,Last 6M,Last 3M,Last 1M
Asset,,,,,,,,,
AAPL,15.33%,-94.83%,-16.41%,-32.70%,23.93%,15.87%,32.51%,16.69%,3.16%
NVDA,-55.02%,-97.61%,-85.74%,-88.96%,-82.77%,-54.28%,-2.22%,34.78%,8.48%


In [6]:
#Saves the dataframe to a csv file. Edit the name
df.to_csv('Optimum 53 63.csv', index=True)